In [ ]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})
import time


# CMPS 2200
# Introduction to Algorithms

## Recurrences


Recurrences are a way to capture the behavior of recursive algorithms.

Key ingredients: 

- Base case ($n = c$): constant time 
- Inductive case ($n > c$): recurse on smaller instance and use output to compute solution

Actually recursion is a conceptual way to view algorithm execution, and we can reframe an algorithm specification to make it recursive.



In [ ]:
def selection_sort(L):
    for i in range(len(L)):
        m = L.index(min(L[i:]))
        L[i], L[m] = L[m], L[i]
    return L
                   
selection_sort([2, 1, 4, 3, 9])

In [ ]:
def selection_sort_recursive(L):
    if (len(L) == 1):
        return(L)
    else:
        m = L.index(min(L))
        L[0], L[m] = L[m], L[0]
        return [L[0]] + selection_sort_recursive(L[1:])
    
selection_sort_recursive([2, 1, 999, 4, 3])

Are these the same algorithm? Can we give a SPARC specification?

<p><span class="math display">\[\begin{array}{l}  
\mathit{selectionsort}~~L = 
\\  
~~~~~\texttt{if}~|L| = 1~\texttt{then}    
\\  
~~~~~~~~~~~~~\texttt{return}~~L  
\\  
~~~~~~~~~\texttt{else}
\\
~~~~~~~~~~~~\texttt{let}\\
~~~~~~~~~~~~~~~m = \texttt{minimum element in}~~L\\
~~~~~~~~~~~~\texttt{in}\\ 
~~~~~~~~~~~~~~~\texttt{Cons}(m, (\mathit{selectionsort~~\langle x | x\in L~~and~~x\neq m})) 
\end{array}\]</span></p>



In [ ]:
What is the running time and why?

$\begin{eqnarray}
T(n) &=& T(n-1) + n \\
 &=& T(n-2) + T(n-1) + 2n-1 \\
&\vdots& \\
&=& \sum_{i=1}^n i  \\
&=& \frac{n(n+1)}{2} \\
&=& \Theta(n^2).
\end{eqnarray}$


Recurrences are a mathematical way to characterize the running time of recursive algorithms.

They are easy to define since they follow the structure of the algorithm.

We'll look at methods to find closed form solutions for recurrence, so that we can obtain big-O bounds for recursive algorithms. 

Let's look at the specification and recurrence for Merge Sort: 

<p><span class="math display">\[\begin{array}{l}  
\mathit{mergeSort}~a =  
\\   
~~~~\texttt{if}~|a| \leq 1~\texttt{then}  
\\   
~~~~~~~~a  
\\  
~~~~\texttt{else}  
\\   
~~~~~~~~\texttt{let}  
\\  
~~~~~~~~~~~~(l,r) = \mathit{splitMid}~a  
\\   
~~~~~~~~~~~~(l',r') = (\mathit{mergeSort}~l \mid\mid{} \mathit{mergeSort}~r)  
\\  
~~~~~~~~\texttt{in}  
\\   
~~~~~~~~~~~~\mathit{merge} (l',r')  
\\  
~~~~~~~~\texttt{end}  
\end{array}\]</span></p>

Suppose that the merging step can be done with $O(n)$ work and $O(\log n)$ span. Then recurrence for the work is: 

$ \begin{equation}
W(n) = \begin{cases}
  c_b, & \text{if $n=1$} \\
  2W(n/2) + c_1n + c_2, & \text{otherwise} 
  \end{cases}
\end{equation}$

How do we solve this recurrence to obtain $W(n) = O(n\log n)$?





![alttext](mergesort_tree.png)

The recursion tree for Merge Sort has linear work at every level except at the leaves. There are a logarithmic number of levels and a linear number of leaves so we obtain an asymptotic bound of $O(n\log n)$ for the work.

What about the span?

![alttext](tree.png)


The recurrence for the span of Mergesort is:

$ \begin{equation}
S(n) = \begin{cases}
  c_3, & \text{if $n=1$} \\
  S(n/2) + c_4 \log n, & \text{otherwise} 
  \end{cases}
\end{equation}$


Since each level of the recursion tree is concurrent and all nodes have the same cost, we have that

$ \begin{align}
S(n) & = & \sum_{i=1}^{(\log n) - 1} \log\frac{n}{2^i}\\
& = & \sum_{i=1}^{(\log n) - 1} (\log n) - i\\
& = & \sum_{i=1}^{(\log n) - 1} (\log n) - \sum_{i=1}^{(\log n) - 1} i\\
& = & \Theta(\log n)\\
\end{align}$


Another recurrence:
    
$ \begin{equation}
W(n) = \begin{cases}
  c_b, & \text{if $n=1$} \\
  2W(n/2) + n^2, & \text{otherwise} 
  \end{cases}
\end{equation}$

What is the asymptotic runtime?

![alttext](tree.png)



Recurrences can be more general, so how do we get a handle on asymptotic runtimes when the recursion is really complicated?

**Key Idea:** The branching properties of the recursion tree determine work at each level and the number of leaves.

The **brick method** gives a way to derive asymptotic runtimes by looking at the relationships between parent and child nodes in the recursion tree. This way we only need to worry about the costs of the root and the leaves.

The value of $n$ decreases geometrically as we collect the terms in our recurrences. We'll make use of bounds for geometric series. For any $\alpha > 0$:
    
$$ \sum_{i=0}^n \alpha^i  = \frac{\alpha}{\alpha - 1}\cdot\alpha^n$$

For $\alpha < 1$:

$$ \sum_{i=0}^\infty \alpha^i  < \frac{1}{1-\alpha}$$



For a node $v$ in the recursion tree, let $C(v)$ denote its cost and $D(v)$ denote its children.

A recurrence is **root-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \geq \alpha \sum_{u \in D(v)} C(u)$$

The cost of a root dominated recurrence is $O(C(r))$ if $r$ is the root.

This is because the cost reduces geometrically as we go toward the leaves, and the total cost bounded by $\alpha/(\alpha-1)$ times $C(r)$.


A recurrence is **leaf-dominated** if for all $v$, there is an $\alpha > 1$ such that:

$$C(v) \leq \frac{1}{\alpha} \sum_{u \in D(v)} C(u)$$

If we have $L$ leaves in the recursion tree, the cost of a leaf dominated recurrence is $O(L)$.

This is because the cost increases geometrically as we go toward the leaves, and the total cost is bounded by $\alpha/(\alpha-1)$ times $c_b \cdot L$.

A recurrence is **balanced** when every level of the recursion tree has the same asymptotic cost. In this case, the recurrence is $O(D(r) \log n) = O(L \log n)$.  


Let's look at some examples:

$$ W(n) = 2 W(n/2) + \sqrt{n} $$

$$ W(n) = 3 W(n/2) + n $$

$$ W(n) = 2 W(n/3) + n $$

$$ W(n) = 3 W(n/3) + n $$


Do you see a way to count the number of leaves in the recursion tree?


>If we have a recurrence of the form $W(n) = aW(n/b) + f(n)$, then the number of leaves is $O(a^{\log_b n})$, or equivanelty, $O(n^{log_b a})$.

More examples (some trickier than others):

$$ W(n) = W(n - 1) + n $$

$$ W(n) = \sqrt{n} W(\sqrt{n}) + n^2 $$

$$ W(n) = W(\sqrt{n}) + W(n/2) + n $$

$$ W(n) = W(n/2) + W(n/3) + 1 $$



Now that we've come up with a general technique for solving recurrences, let's look at a recursive algorithm. You learned this algorithm in elementary school for integer multiplication:

- Input: $n$ bit integers $x= \langle x_{n-1}, \ldots, x_0\rangle$ and $y = \langle y_{n-1}, \ldots, y_0\rangle$

- Output: $x \cdot y$

- Example: '1001'$\times$'1101'


What is the running time of the "elementary school" algorithm?

>For two $n$-digit inputs $O(n^2)$, since for each digit of $x$ we might add a stack of $n$ bits. The total number of bits in the solution is $2n$.

What does this have to do with recursion and recurrences?

Instead of the elementary school algorithm, consider splitting each $n$-digit input in half. Can we multiply recursively?

Let $x = \langle x_L, x_R\rangle$, $y = \langle y_L, y_R\rangle$. Then,

$\begin{align} 
x &=& 2^{n/2} x_L + x_R \\
y &=& 2^{n/2} y_L + y_R \\
\end{align}
$

So then,

$\begin{align}
x\cdot y &=& (2^{n/2} x_L + x_R)(2^{n/2} y_L + y_R) \\
 &=& 2^n x_L \cdot y_L + 2^{n/2} (x_L \cdot y_R + x_R \cdot y_L) + x_R \cdot y_R \\
\end{align}
$

What recursive algorithm, and recurrence is suggested by this observation?

>$W(n) = 4W(n/2) + cn$

What is the solution to this recurrence using the brick method? Is it root-dominated, or leaf-dominated?

> $O(n^2)$, the same as the elementary school algorithm!



Can we do better?

Observation:
    
$\begin{align} 
(x_L + x_R)\cdot (y_L + y_R) &=& x_L\cdot y_L + x_L\cdot y_R + x_R\cdot y_L + x_R\cdot y_L\\
\end{align}
$

Recall that 

$\begin{align}
x\cdot y &=& (2^{n/2} x_L + x_R)(2^{n/2} y_L + y_R) \\
 &=& 2^n x_L \cdot y_L + 2^{n/2} (x_L \cdot y_R + x_R \cdot y_L) + x_R \cdot y_R \\
\end{align}
$


How does our observation help us?

If we calculate $x_L\cdot y_L$, $x_R\cdot y_R$, and $(x_L + x_R)\cdot (y_L + y_R)$, that is *three* recursive multiplications. 

We need 4 terms for the solution, but notice that 

$\begin{align}
x_L\cdot y_R + x_R\cdot y_L = (x_L + x_R)\cdot (y_L + y_R) - x_L\cdot y_L - x_R\cdot y_R\\
\end{align}$

So with 3 recursive multiplications and two more "additions", we then get that $W(n) = 3W(n/2) + dn$. What is the running time?

Using the brick method, this is still a leaf-dominated recurrence, and thus the running time is $O(n^{\log_2 3})$ instead of $O(n^2)$.

This is known as the **Karatsaba-Ofman** algorithm, and is widely used in hardware!